In [ ]:
! pip install zemberek-python
! curl https://huggingface.co/datasets/husnu/tquad2/raw/main/tquad_train_data_v2.json > tquad_train_data_v2.json
! curl https://huggingface.co/datasets/husnu/tquad2/raw/main/tquad_dev_data_v2.json > tquad_dev_data_v2.json

In [1]:
import math
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import pytorch_lightning as pl

from transformers import MT5ForConditionalGeneration, MT5TokenizerFast

from zemberek import TurkishSentenceExtractor


In [2]:
def tquad2df(path):
    extractor = TurkishSentenceExtractor()

    df = {'title': [], 'context': [], 'question': [], 'cloze': [], 'answer': []}

    dataset = pd.read_json(path).data


    for data in dataset:
        title = data['title']
        for para in data['paragraphs']:
            context = para['context']
            for qa in para['qas']:
                question = qa['question']

                unique_answers = set()
                for answer in qa['answers']:
                    answer_text, answer_span = answer['text'], int(answer['answer_start'])
                    spans = extractor.extract_to_spans(context)

                    for span in spans:
                        if answer_text not in unique_answers and span.in_span(answer_span):
                            unique_answers.add(answer_text)
                            cloze = span.get_sub_string(context)
                            df['title'].append(title)
                            df['context'].append(context)
                            df['question'].append(question)
                            df['cloze'].append(cloze)
                            df['answer'].append(answer_text)

    return pd.DataFrame(df)


In [3]:
def create_text(**kwargs):
    answer = kwargs['answer']
    cloze = kwargs['cloze']
    return f"generate question for answer {answer} : {cloze}"

In [4]:
class TData(Dataset):
    def __init__(self, df, tokenizer):
        super(TData, self).__init__()

        self.df = df
        self.tok = tokenizer

    def __getitem__(self, i):
        row = self.df.iloc[i]

        text = create_text(answer=row['answer'], cloze=row['cloze'])

        model_inputs = self.tok(text, padding='max_length', max_length=256,
                                truncation=True, return_tensors='pt')
        with self.tok.as_target_tokenizer():
            labels = self.tok(row['question'], padding='max_length', max_length=256,
                                truncation=True, return_tensors="pt")
        model_inputs["labels"] = labels["input_ids"]

        return {k: v[0] for k, v in model_inputs.items()}

    def __len__(self):
        return len(self.df)

In [6]:
TRAIN_DIR = '../../taboo-datasets/tquad2/tquad_train_data_v2.json'
DEV_DIR= '../../taboo-datasets/tquad2/tquad_dev_data_v2.json'

train_df = tquad2df(TRAIN_DIR)
val_df = tquad2df(DEV_DIR)

In [11]:
train_df.iloc[0].question

'Kim geldiğinde orijinal viking yerleşimcilerine ortak bir kimlik vermiştir?'

In [6]:
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small', force_download=True)
tokenizer = MT5TokenizerFast.from_pretrained('google/mt5-small')

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [7]:
train_data = TData(train_df, tokenizer)
val_data = TData(val_df, tokenizer)

In [8]:
class AttackerModel(pl.LightningModule):
    def __init__(self, model, lr):
        super(AttackerModel, self).__init__()

        self.model = model
        self.lr = lr

    def forward(self, **batch):
        return self.model(**batch)

    def training_step(self, batch, batch_idx):
        loss = self(**batch).loss
        self.log_dict({'loss': loss, 'ppl': math.exp(loss.item())}, 
                      prog_bar=True, on_step=True, on_epoch=True)

        if batch_idx % 500 == 0:
            with torch.no_grad():
                sentence = train_df.iloc[0]
                text = create_text(**sentence)
                tokenized_sent = tokenizer(text, padding='max_length', max_length=256,
                                           truncation=True, return_tensors='pt')
                generated_question = self.model.cuda().generate(tokenized_sent['input_ids'].cuda(), max_length=256, do_sample=True, top_k=50,
                                                                      top_p=0.95, num_beams=5, num_return_sequences=3)
                print(tokenizer.batch_decode(generated_question, skip_special_tokens=True))

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self(**batch).loss
        self.log_dict({'loss': loss, 'ppl': math.exp(loss.item())}, 
                      prog_bar=True, on_step=True, on_epoch=True)

    def configure_optimizers(self):
        return optim.AdamW(self.model.parameters())


In [9]:
BATCH_SIZE = 8
LR = 1e-3
EPOCHS = 15

In [10]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

In [ ]:
attacker_model = AttackerModel(model=model, lr=LR)
trainer = pl.Trainer(accelerator='gpu', devices=1, max_epochs=EPOCHS)
trainer.fit(model=attacker_model, train_dataloaders=train_loader, val_dataloaders=val_loader)

2023-02-28 06:25:45,057 - pytorch_lightning.utilities.rank_zero - INFO
Msg: GPU available: True (cuda), used: True

2023-02-28 06:25:45,058 - pytorch_lightning.utilities.rank_zero - INFO
Msg: TPU available: False, using: 0 TPU cores

2023-02-28 06:25:45,059 - pytorch_lightning.utilities.rank_zero - INFO
Msg: IPU available: False, using: 0 IPUs

2023-02-28 06:25:45,061 - pytorch_lightning.utilities.rank_zero - INFO
Msg: HPU available: False, using: 0 HPUs

2023-02-28 06:25:46,821 - pytorch_lightning.accelerators.cuda - INFO
Msg: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

2023-02-28 06:25:46,834 - pytorch_lightning.callbacks.model_summary - INFO
Msg: 
  | Name  | Type                        | Params
------------------------------------------------------
0 | model | MT5ForConditionalGeneration | 300 M 
------------------------------------------------------
300 M     Trainable params
0         Non-trainable params
300 M     Total params
1,200.707 Total estimated model params size (MB)



Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

["<extra_id_0> '-a", '<extra_id_0>igma)==  <extra_id_4>hinja <extra_id_33>нукле <extra_id_31>“ <extra_id_31> vulner <extra_id_31> <extra_id_31>улан्बाντανக்கீאוזασκෝරෝර <extra_id_44>-возбуд <extra_id_33>สมาชิก dispuෝලข้อ現行აჟ <extra_id_39>rzejිනාղբпне哔 <extra_id_31>ნაწ@ <extra_id_34> Altersintari <extra_id_33>', 'saanоđграде🖏オン <extra_id_54>იტეტReplyDeleteAddpratsiyon']
['<extra_id_0>', '<extra_id_0>?', '<extra_id_0>']
['<extra_id_0> ', 'Ne', '<extra_id_0>']
["' y?? ne   ? ???", 'Kim   ', '<extra_id_0>.?']


Validation: 0it [00:00, ?it/s]

...

["İbn Rüşd'in ölüm sebebi nedir?", 'II. Mehmed nerede doğmuştur?', "II. Bayezid'in saltanatı kimdir?"]
['Hangi antlaşmadır?', 'Hangi antlaşmaya göre hangi antlaşmadır?', "İbn-i Batuta'nın hangi ülkede yapıldı?"]
["I. Mahmud'ın babası kimdir?", 'Kimin babası kimdir?', "İbn-i Batuta'nın adı nedir?"]
['II. Bayezid hangi tarihte ölmüştür?', "II. Mustafa'nın yeğeni Feyzullah Efendi'nin neyi ele geçirmiştir?", "II. Mustafa'nın babası kimdir?"]


Validation: 0it [00:00, ?it/s]

["II. Mehmed'in babası kimdir?", "I. Bayezid'den sonra gelen padişahın adı nedir?", "I. Süleyman'ın babası kimdir?"]
['Şehzade Selim nerede doğmuştur?', "II. Mehmed'in saltanatı kaç yılında ölmüştür?", 'Kim, divan edebiyatındaki mahlası kimdir?']
["İbn Tağrıberdî'nin ölüm sebebi nedir?", "Aziz Sancar'ın babası kimdir?", "II. Bayezid'in saltanatı kaç yılında başlamıştır?"]
["İbn-i Batuta'nın yazarı kimdir?", "İbn-i Batuta'nın yazarı kimdir?", 'Aydın Sayılı hangi bölümden mezun olmuştur?']


Validation: 0it [00:00, ?it/s]

["İbn-i Batuta'nın asıl adı nedir?", "Şehzade Bayezid'in babası kimdir?", "Aydın Sayılı'nın doğum yeri neresidir?"]
['II. Bayezid hangi tarihte ölmüştür?', "II. Bayezid'in ikinci sefer-i kime aittir?", 'II. Bayezid hangi tarihte ölmüştür?']
